In [25]:
import pandas as pd
import os
import pandas as pd
import requests
import sys
import xarray as xr
# import dates as date

comid = 1479611
gauge_name = 'JAIT2'
nwm_discharge = pd.read_csv('D:/Sujana/Project/NWM_discharge/discharge_'+str(comid)+'.csv')
dd6_data = pd.read_csv(r'D:/Sujana/Project/DD6_CSV/DD6_'+gauge_name+'.csv')
nwm_discharge['Time']=pd.to_datetime(nwm_discharge['Time'])
dd6_data['timestamp']=pd.to_datetime(dd6_data['timestamp'])
# nwm_discharge = nwm_discharge.set_index('Time') 
dd6_data = dd6_data.set_index('timestamp')




In [27]:
nwm_discharge.head()

,Unnamed: 0,Time,Discharge_NWM
0,0,2019-09-17 00:00:00,0.26
1,1,2019-09-17 01:00:00,0.25
2,2,2019-09-17 02:00:00,0.25
3,3,2019-09-17 03:00:00,0.25
4,4,2019-09-17 04:00:00,0.25


In [28]:
dd6_data.head()

,value,quality
timestamp,,
2019-09-17 00:23:42+00:00,2.82,NaN
2019-09-17 08:53:43+00:00,2.92,NaN
2019-09-17 09:08:44+00:00,2.87,NaN
2019-09-17 09:23:43+00:00,2.97,NaN
2019-09-17 12:38:44+00:00,3.12,NaN


In [29]:
dd6_data.index.get_loc(nwm_discharge['Time'], method='nearest')

InvalidIndexError: 0     2019-09-17 00:00:00
1     2019-09-17 01:00:00
2     2019-09-17 02:00:00
3     2019-09-17 03:00:00
4     2019-09-17 04:00:00
              ...        
259   2019-09-27 19:00:00
260   2019-09-27 20:00:00
261   2019-09-27 21:00:00
262   2019-09-27 22:00:00
263   2019-09-27 23:00:00
Name: Time, Length: 264, dtype: datetime64[ns]